<a href="https://colab.research.google.com/github/omerhac/arc_challenge/blob/master/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data loading


In [ ]:
def load_data():
  """
  Loads all the data into training_tasks, eval_tasks and test_tasks
  """

  ## get paths
  GCS_PATH = "gs://kds-d3cfb3d523ca35d2517017a78110126404d01fdea69417ce49950459"
  training_filenames = tf.io.gfile.glob(GCS_PATH + "/training/*")
  test_filenames = tf.io.gfile.glob(GCS_PATH + "/test/*")
  eval_filenames = tf.io.gfile.glob(GCS_PATH + "/evaluation/*")

  # create datasets with filenames
  training_dataset = tf.data.Dataset.list_files(training_filenames)
  eval_dataset = tf.data.Dataset.list_files(eval_filenames)
  test_dataset = tf.data.Dataset.list_files(test_filenames)

  # load the jsons
  def load_task(filename):
    task_json = tf.io.read_file(filename)
    return task_json

  training_dataset = training_dataset.map(load_task)
  eval_dataset = eval_dataset.map(load_task)
  test_dataset = test_dataset.map(load_task)

  training_dataset_numpy = tf.data.Dataset.as_numpy_iterator(training_dataset) # convert to numpy iterator
  eval_dataset_numpy = tf.data.Dataset.as_numpy_iterator(eval_dataset)
  test_dataset_numpy = tf.data.Dataset.as_numpy_iterator(test_dataset)

  ## create a numpy array of tasks (n_tasks, )
  def list_from_jsons(jsons_numpy_iterator):
    """
      Create a list of task dictionaries from jsons numpy interator
    """

    tasks = []
    for task in jsons_numpy_iterator:
      tasks.append(json.loads(task))

    return tasks

  ## get numpy arrays of datasets
  training_tasks = list_from_jsons(training_dataset_numpy)
  eval_tasks = list_from_jsons(eval_dataset_numpy)
  test_tasks = list_from_jsons(test_dataset_numpy)

  return training_tasks, eval_tasks, test_tasks


def load_data_from_jsons():
  """
  Load tasks from jsons to lists of tasks.

  Returns:
  training_tasks, eval_tasks, test_tasks --> lists of tasks
  """

  with open("training_tasks.json", 'r') as f:
    training_tasks = json.load(f)

  with open("eval_tasks.json", 'r') as f:
    eval_tasks = json.load(f)
  
  with open("test_tasks.json", 'r') as f:
    test_tasks = json.load(f)

  return training_tasks, eval_tasks, test_tasks

# Utils

In [ ]:
def plot_board(board, ax, title=""):
  """
  Plot a board on a given axis
  """
  cmap = colors.ListedColormap(
      ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
        '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
  norm = colors.Normalize(vmin=0, vmax=9)
  
  ax.imshow(board, cmap=cmap, norm=norm)
  ax.grid(True,which='both',color='lightgrey', linewidth=0.5)    
  ax.set_yticks([x-0.5 for x in range(1+board.shape[0])])
  ax.set_xticks([x-0.5 for x in range(1+board.shape[1])])     
  ax.set_xticklabels([])
  ax.set_yticklabels([])
  ax.set_title(title)

def plot_one(task, ax, i,train_or_test,input_or_output):
  """
  Plot one task on a given axis
  """
  cmap = colors.ListedColormap(
      ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
        '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
  norm = colors.Normalize(vmin=0, vmax=9)
  
  input_matrix = task[train_or_test][i][input_or_output]
  ax.imshow(input_matrix, cmap=cmap, norm=norm)
  ax.grid(True,which='both',color='lightgrey', linewidth=0.5)    
  ax.set_yticks([x-0.5 for x in range(1+len(input_matrix))])
  ax.set_xticks([x-0.5 for x in range(1+len(input_matrix[0]))])     
  ax.set_xticklabels([])
  ax.set_yticklabels([])
  ax.set_title(train_or_test + ' '+input_or_output)
    

def plot_task(task):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """    
    num_train = len(task['train'])
    fig, axs = plt.subplots(2, num_train, figsize=(3*num_train,3*2))
    for i in range(num_train):     
        plot_one(task, axs[0,i],i,'train','input')
        plot_one(task, axs[1,i],i,'train','output')        
    plt.tight_layout()
    plt.show()        
        
    num_test = len(task['test'])
    fig, axs = plt.subplots(2, num_test, figsize=(3*num_test,3*2))
    if num_test==1: 
        plot_one(task, axs[0],0,'test','input')
        plot_one(task, axs[1],0,'test','output')     
    else:
        for i in range(num_test):      
            plot_one(task, axs[0,i],i,'test','input')
            plot_one(task, axs[1,i],i,'test','output')  
    plt.tight_layout()
    plt.show() 
  

def plot_board_pairs(board_pairs, labels):
  """
  Plots the board pairs (for siamese networks) with their label as a title
  """

  fig, axs = plt.subplots(len(board_pairs), 2, figsize=(8, 3 * len(board_pairs)))
  
  for i, pair in enumerate(board_pairs):
    # plot a pair on a given axis
    plot_board(pair[0], axs[i, 0], title="anchor") 
    plot_board(pair[1], axs[i, 1], title=labels[i])
  
  plt.tight_layout()
  plt.show()


def plot_decoder_boards(board_pairs):
  """
  Plots the board pairs (for siamese networks) with their label as a title
  """

  fig, axs = plt.subplots(len(board_pairs), 2, figsize=(8, 3 * len(board_pairs)))

  for i, pair in enumerate(board_pairs):
    # plot a pair on a given axis
    plot_board(pair[0], axs[i, 0]) 
    plot_board(pair[1], axs[i, 1])

  plt.tight_layout()
  plt.show()


def display_training_curves(hist, metric='accuracy', with_val=False):
  """display learning curves for keras history dict, args: history dict, with val --> boolean with/without val"""
  plt.figure(figsize=(18,6))

  # accuracy plots
  plt.subplot(1,2,1)
  plt.plot(hist[metric])
  
  if with_val:
    plt.plot(hist['val_' + metric])
    plt.legend(['Train', 'Validation'])
  
  else:
    plt.legend(['Train'])
  
  plt.title('Model accuracy')
  plt.xlabel('EPOCH')
  plt.ylabel('Accuracy')

  # loss plots
  plt.subplot(1,2,2)
  plt.plot(hist['loss'])

  if with_val:
    plt.plot(hist['val_loss'])
    plt.legend(['Train loss', 'Val loss'])
  
  else:
    plt.legend(['Train loss'])
  
  plt.title('Model loss')
  plt.xlabel('EPOCH')
  plt.ylabel('Loss')
  plt.show()